# Projeto Final - AMAP - 2024.2

## 1. Importação das Bibliotecas

In [9]:
import sys
sys.path.append("..")
import torch
from torch.utils.data import DataLoader
from torch.optim.adam import Adam
import matplotlib.pyplot as plt
#from src.models import vae
#from src.data.dataloader import load_data

In [15]:
from src.utils.metrics import VAELoss
from src.utils.utils import train, save_sample
from src.data.data import LoFiDataset
from src.models.vae import DenseVAE, Conv1DVAE

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


## 2. Carregamento dos Dados

In [19]:
dataset = LoFiDataset('../data/raw')

In [10]:
batch_size = 32
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

## 3. Instanciação do Modelo

In [13]:
model = Conv1DVAE(2, 2, 32921, 2, 400, 100, 4, 2, device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
loss_function = VAELoss()

In [14]:
model

Conv1DVAE(
  (encoder): Conv1DEncoder(
    (block): Sequential(
      (0): Conv1d(1, 4, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): LeakyReLU(negative_slope=0.2)
      (2): Conv1d(4, 8, kernel_size=(3,), stride=(2,), padding=(1,))
      (3): LeakyReLU(negative_slope=0.2)
      (4): Flatten(start_dim=1, end_dim=-1)
      (5): DenseEncoder(
        (block): Sequential(
          (0): Linear(in_features=65848, out_features=400, bias=True)
          (1): LeakyReLU(negative_slope=0.2)
          (2): Linear(in_features=400, out_features=100, bias=True)
          (3): LeakyReLU(negative_slope=0.2)
        )
      )
    )
  )
  (mean_layer): Linear(in_features=100, out_features=2, bias=True)
  (logvar_layer): Linear(in_features=100, out_features=2, bias=True)
  (decoder): Conv1DDecoder(
    (block): Sequential(
      (0): DenseDecoder(
        (block): Sequential(
          (0): Linear(in_features=2, out_features=100, bias=True)
          (1): LeakyReLU(negative_slope=0.2)
        

In [18]:
loss_curve = train(model, train_loader, loss_function, optimizer, epochs=100, device=device, save_path='best_model.pth')

In [ ]:
plt.plot(loss_curve)

In [ ]:
model.load_state_dict(torch.load('best_model.pth', weights_only=True))

In [ ]:
a = model.sample(scale=2)
b = save_sample(a, 'asd.wav')
plt.plot(b[0])